# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
! pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [3]:
# loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true')

# loans

loans = pd.read_csv(
    'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true',
    compression='gzip',
    encoding='utf-8'
)

loans.to_csv('loans.csv')


/tmp/ipykernel_7375/3955335309.py:5: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv(


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [4]:
# column_types =
columns = loans.columns.to_list()
print(columns)

loans.dtypes.unique()

['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_d

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [5]:
! pip install openpyxl


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [7]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [8]:
# COdigo guardar
# Guardar el DataFrame en formato pickle
datos_dict.to_pickle('datos_dict.pkl')


In [9]:
# Codigo para cargar
# Cargar el DataFrame desde el archivo pickle
datos_dict_cargado = pd.read_pickle('datos_dict.pkl')


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [10]:
loans.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [11]:
# Manejos de tipos 1
# Tu codigo aqui

int64_columns = [col for col in loans.columns if loans[col].dtype == 'int64']

def change_cast_to_int32(df, columns):
    int32_min, int32_max = -2**31, 2**31 - 1  # Range of int32
    
    for col in columns:
        if col not in df.columns:
            print(f"Column '{col}' does not exist in the DataFrame.")
            continue

        # Check if column is numeric
        if not pd.api.types.is_numeric_dtype(df[col]):
            print(f"Column '{col}' is not numeric, skipping.")
            continue

        # Check for NaNs and analyze range
        has_nan = df[col].isna().any()
        min_val = df[col].min()
        max_val = df[col].max()

        print(f"\nAnalyzing column '{col}':")
        print(f"  - Min value: {min_val}")
        print(f"  - Max value: {max_val}")
        print(f"  - Contains NaNs: {'Yes' if has_nan else 'No'}")

        # Evaluate if casting to int32 is safe
        if has_nan:
            print(f"  - Column '{col}' has NaN values, conversion to int32 will not be possible without handling them.")
        elif min_val >= int32_min and max_val <= int32_max:
            print(f"  - Column '{col}' is within int32 range. It can be safely cast to int32.")
            # Optional: Cast to int32 if suitable
            df[col] = df[col].astype('int32')
            print(f"  - Column '{col}' has been cast to int32.")
        else:
            print(f"  - Column '{col}' has values outside int32 range. Conversion to int32 is not safe.")



In [12]:
loans[int64_columns]

,Unnamed: 0,id
0,0,38098114
1,1,36805548
2,2,37842129
3,3,37612354
4,4,37662224
...,...,...
99995,99995,22454240
99996,99996,11396920
99997,99997,8556176
99998,99998,24023408


In [13]:
change_cast_to_int32(loans, int64_columns)


Analyzing column 'Unnamed: 0':
  - Min value: 0
  - Max value: 99999
  - Contains NaNs: No
  - Column 'Unnamed: 0' is within int32 range. It can be safely cast to int32.
  - Column 'Unnamed: 0' has been cast to int32.

Analyzing column 'id':
  - Min value: 57167
  - Max value: 38098114
  - Contains NaNs: No
  - Column 'id' is within int32 range. It can be safely cast to int32.
  - Column 'id' has been cast to int32.


In [14]:
# Manejos de tipos 2
# Tu codigo aqui

float64_columns = [col for col in loans.columns if loans[col].dtype == 'float64']

def cast_float32(df, columns):
    float32_min, float32_max = np.finfo(np.float32).min, np.finfo(np.float32).max  # Range of float32
    
    for col in columns:
        if col not in df.columns:
            print(f"Column '{col}' does not exist in the DataFrame.")
            continue

        # Check if column is of type float64
        if df[col].dtype != 'float64':
            print(f"Column '{col}' is not of type float64, skipping.")
            continue

        # Check for NaNs and analyze range
        has_nan = df[col].isna().any()
        min_val = df[col].min()
        max_val = df[col].max()

        print(f"\nAnalyzing column '{col}':")
        print(f"  - Min value: {min_val}")
        print(f"  - Max value: {max_val}")
        print(f"  - Contains NaNs: {'Yes' if has_nan else 'No'}")

        # Evaluate if casting to float32 is safe
        if min_val >= float32_min and max_val <= float32_max:
            print(f"  - Column '{col}' is within float32 range and can be safely cast to float32.")
            # Optional: Cast to float32 if suitable
            df[col] = df[col].astype('float32')
            print(f"  - Column '{col}' has been cast to float32.")
        else:
            print(f"  - Column '{col}' has values outside the float32 range. Conversion to float32 is not safe.")


In [15]:
loans[float64_columns]

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,NaN,15000.0,15000.0,15000.0,12.39,336.64,78000.0,12.03,0.0,750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,10400.0,10400.0,10400.0,6.99,321.08,58000.0,14.92,0.0,710.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,21425.0,21425.0,21425.0,15.59,516.36,63800.0,18.49,0.0,685.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,12800.0,12800.0,12800.0,17.14,319.08,125000.0,8.31,1.0,665.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,7650.0,7650.0,7650.0,13.66,260.20,50000.0,34.81,0.0,685.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,8400.0,8400.0,8400.0,9.17,267.79,91000.0,7.78,1.0,705.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,NaN,10000.0,10000.0,10000.0,12.99,336.90,53133.0,15.25,0.0,690.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,NaN,30000.0,30000.0,30000.0,20.99,811.44,78000.0,23.75,0.0,665.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,NaN,8475.0,8475.0,8475.0,24.99,336.92,52000.0,17.61,2.0,660.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
cast_float32(loans, float64_columns)


Analyzing column 'member_id':
  - Min value: nan
  - Max value: nan
  - Contains NaNs: Yes
  - Column 'member_id' has values outside the float32 range. Conversion to float32 is not safe.

Analyzing column 'loan_amnt':
  - Min value: 1000.0
  - Max value: 35000.0
  - Contains NaNs: No
  - Column 'loan_amnt' is within float32 range and can be safely cast to float32.
  - Column 'loan_amnt' has been cast to float32.

Analyzing column 'funded_amnt':
  - Min value: 1000.0
  - Max value: 35000.0
  - Contains NaNs: No
  - Column 'funded_amnt' is within float32 range and can be safely cast to float32.
  - Column 'funded_amnt' has been cast to float32.

Analyzing column 'funded_amnt_inv':
  - Min value: 1000.0
  - Max value: 35000.0
  - Contains NaNs: No
  - Column 'funded_amnt_inv' is within float32 range and can be safely cast to float32.
  - Column 'funded_amnt_inv' has been cast to float32.

Analyzing column 'int_rate':
  - Min value: 6.0
  - Max value: 26.06
  - Contains NaNs: No
  - Colum

In [17]:
object_columns = [col for col in loans.columns if loans[col].dtype == 'O']

loans[object_columns]

for i in object_columns:
    if 'flag' in i:
        print(i)

loans[['hardship_flag', 'debt_settlement_flag']]


hardship_flag
debt_settlement_flag
debt_settlement_flag_date


,hardship_flag,debt_settlement_flag
0,N,N
1,N,N
2,N,N
3,N,N
4,N,N
...,...,...
99995,N,N
99996,N,N
99997,N,N
99998,N,N


In [18]:
bool_cols = ['hardship_flag', 'debt_settlement_flag']

for col in bool_cols:
    loans[col] = loans[col].apply(lambda x: True if x == 'Y' else (False if x == 'N' else None)).astype('boolean')

#igual encontré que pymnt_plan esta en el formato de y y n
loans['pymnt_plan'] = loans['pymnt_plan'].apply(lambda x: True if x == 'Y' else (False if x == 'N' else None)).astype('boolean')

In [19]:
nan_only_columns = [col for col in loans.columns if loans[col].isna().all()]
nan_only_columns

loans[nan_only_columns]

,member_id,pymnt_plan,annual_inc_joint,dti_joint,verification_status_joint,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,...,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#cambiar grades y subgrades a categorical data porque son categorias
loans['grade'] = loans['grade'].astype('category')
loans['sub_grade'] = loans['sub_grade'].astype('category')

#de igual manera home_ownership, loan_status, verification_status, purpose, initial_list_status, policy_code, application_type, hardship_status, hardship_type
#porque tienen valores repetidos
loans['home_ownership'] = loans['home_ownership'].astype('category')
loans['loan_status'] = loans['loan_status'].astype('category')
loans['verification_status'] = loans['verification_status'].astype('category')
loans['purpose'] = loans['purpose'].astype('category') 
loans['initial_list_status'] = loans['initial_list_status'].astype('category')
loans['policy_code'] = loans['policy_code'].astype('category')
loans['application_type'] = loans['application_type'].astype('category')
loans['hardship_status'] = loans['hardship_status'].astype('category')
loans['hardship_type'] = loans['hardship_type'].astype('category')

#Cambiamos
loans['desc'] = loans['desc'].astype(str) 
loans['title'] = loans['title'].astype(str)
loans['zip_code'] = loans['zip_code'].astype(str)  
loans['addr_state'] = loans['addr_state'].astype(str)

In [24]:
# Convert columns to datetime type
date_columns = [
    'issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d',
    'last_credit_pull_d', 'hardship_start_date', 'hardship_end_date',
    'payment_plan_start_date', 'debt_settlement_flag_date', 'settlement_date'
]
for col in date_columns:
    loans[col] = pd.to_datetime(loans[col], errors='coerce')

/tmp/ipykernel_7375/3952976714.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[col] = pd.to_datetime(loans[col], errors='coerce')
/tmp/ipykernel_7375/3952976714.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[col] = pd.to_datetime(loans[col], errors='coerce')
/tmp/ipykernel_7375/3952976714.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[col] = pd.to_datetime(loans[col], errors='coerce')
/tmp/ipykernel_7375/3952976714.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure par

In [21]:
loans['desc']

0        nan
1        nan
2        nan
3        nan
4        nan
        ... 
99995    nan
99996    nan
99997    nan
99998    nan
99999    nan
Name: desc, Length: 100000, dtype: object

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [25]:
import pandas as pd
#primero quitamos todas las columnas que solo contengan Nan porque son columnas que no aportan nada de información y ocupan espacio
def drop_nan_only_columns(df):
    """
    Drops columns that contain only NaN values.

    Parameters:
    df (pd.DataFrame): The DataFrame from which to drop NaN-only columns.

    Returns:
    pd.DataFrame: A DataFrame with NaN-only columns removed.
    """
    # Drop columns where all values are NaN
    df_cleaned = df.dropna(axis=1, how='all')
    return df_cleaned

# Example usage
# Assuming `loans` is your DataFrame
loans_cleaned = drop_nan_only_columns(loans)

In [34]:
import pandas as pd
import numpy as np
import json

# Function to handle numeric column imputation
def imputar_numerico(df, column):
    # Use mean or median based on skewness
    if abs(df[column].skew()) < 1:
        value = df[column].mean()
        strategy = 'mean'
    else:
        value = df[column].median()
        strategy = 'median'
    df[column].fillna(value)
    return {"columna":column,"estrategia": strategy, "valor": value}

In [35]:
def imputar_categorico(df, column):
    # Use a placeholder string for missing values
    value = "missing"
    df[column].fillna(value)
    return { "estrategia": "identificador", "valor": value}

In [36]:
# Function to handle date column imputation
def imputar_fecha(df, column):
    # Use a placeholder date for missing values
    value = pd.Timestamp("1900-01-01")
    df[column].fillna(value)
    return {"estrategia": "placeholder_date", "valor": str(value)}

In [37]:
# Function to handle boolean column imputation
def imputar_booleano(df, column):
    # Use False as the default for missing values
    value = False
    df[column].fillna(value)
    return {"estrategia": "default_bool", "valor": value}

In [42]:

# Main function to apply imputations based on data type
def imputar_valores(df):
    imputation_strategies = {}
    for column in df.columns:
        if df[column].isna().sum() > 0:
            print("woo")
            if pd.api.types.is_numeric_dtype(df[column]):
                imputation_strategies[column] = imputar_numerico(df, column)
            elif pd.api.types.is_object_dtype(df[column]):
                imputation_strategies[column] = imputar_categorico(df, column)
            elif pd.api.types.is_datetime64_any_dtype(df[column]):
                imputation_strategies[column] = imputar_fecha(df, column)
            elif pd.api.types.is_bool_dtype(df[column]):
                imputation_strategies[column] = imputar_booleano(df, column)
        else:
            imputation_strategies[column] = {"estrategia": "sin_imputacion", "valor": None}
    return imputation_strategies



In [39]:
explanation_strategies = {
    "mean": "Replaces NaN values with the mean of the column for balanced data.",
    "median": "Replaces NaN values with the median of the column for skewed data.",
    "identificador": "Replaces NaN values in categorical columns with a placeholder like 'missing'.",
    "placeholder_date": "Replaces NaN values in date columns with a default date (e.g., '1900-01-01').",
    "default_bool": "Replaces NaN values in boolean columns with a default False value.",
    "sin_imputacion": "Indicates no imputation was needed because no NaN values were present."
}

In [43]:
imputation_strategies = imputar_valores(loans_cleaned)

woo
woo


In [41]:
imputation_strategies

{'Unnamed: 0': {'estrategia': 'sin_imputacion', 'valor': None},
 'id': {'estrategia': 'sin_imputacion', 'valor': None},
 'loan_amnt': {'estrategia': 'sin_imputacion', 'valor': None},
 'funded_amnt': {'estrategia': 'sin_imputacion', 'valor': None},
 'funded_amnt_inv': {'estrategia': 'sin_imputacion', 'valor': None},
 'term': {'estrategia': 'sin_imputacion', 'valor': None},
 'int_rate': {'estrategia': 'sin_imputacion', 'valor': None},
 'installment': {'estrategia': 'sin_imputacion', 'valor': None},
 'grade': {'estrategia': 'sin_imputacion', 'valor': None},
 'sub_grade': {'estrategia': 'sin_imputacion', 'valor': None},
 'emp_title': {'estrategia': 'sin_imputacion', 'valor': None},
 'emp_length': {'estrategia': 'sin_imputacion', 'valor': None},
 'home_ownership': {'estrategia': 'sin_imputacion', 'valor': None},
 'annual_inc': {'estrategia': 'sin_imputacion', 'valor': None},
 'verification_status': {'estrategia': 'sin_imputacion', 'valor': None},
 'issue_d': {'estrategia': 'sin_imputacion',

### Codigo para salvar y cargar JSONs

In [44]:
# Save imputation strategies to JSON
with open("imputation_strategies.json", "w") as file:
    json.dump(imputation_strategies, file, indent=4)

# Save strategy explanations to JSON
with open("strategy_explanations.json", "w") as file:
    json.dump(explanation_strategies, file, indent=4)

In [45]:
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Load the JSON containing imputation strategies for each column
imputation_strategies = load_json("imputation_strategies.json")
print("Imputation Strategies:\n", imputation_strategies)

# Load the JSON containing explanations of each strategy
strategy_explanations = load_json("strategy_explanations.json")
print("\nStrategy Explanations:\n", strategy_explanations)

Imputation Strategies:
 {'Unnamed: 0': {'estrategia': 'sin_imputacion', 'valor': None}, 'id': {'estrategia': 'sin_imputacion', 'valor': None}, 'loan_amnt': {'estrategia': 'sin_imputacion', 'valor': None}, 'funded_amnt': {'estrategia': 'sin_imputacion', 'valor': None}, 'funded_amnt_inv': {'estrategia': 'sin_imputacion', 'valor': None}, 'term': {'estrategia': 'sin_imputacion', 'valor': None}, 'int_rate': {'estrategia': 'sin_imputacion', 'valor': None}, 'installment': {'estrategia': 'sin_imputacion', 'valor': None}, 'grade': {'estrategia': 'sin_imputacion', 'valor': None}, 'sub_grade': {'estrategia': 'sin_imputacion', 'valor': None}, 'emp_title': {'estrategia': 'sin_imputacion', 'valor': None}, 'emp_length': {'estrategia': 'sin_imputacion', 'valor': None}, 'home_ownership': {'estrategia': 'sin_imputacion', 'valor': None}, 'annual_inc': {'estrategia': 'sin_imputacion', 'valor': None}, 'verification_status': {'estrategia': 'sin_imputacion', 'valor': None}, 'issue_d': {'estrategia': 'sin_imp